<a href="https://colab.research.google.com/github/Shivayogi-A/Pyspark_programming/blob/master/CASE_in_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

#creating spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("CaseInPyspark")\
        .getOrCreate()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (180 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

**PySpark SQL Case When on DataFrame.**\
If you have a SQL background you might have familiar with Case When statement that is used to execute a sequence of conditions and returns a value when the first condition met, similar to SWITH and IF THEN ELSE statements. Similarly, PySpark SQL Case When statement can be used on DataFrame, below are some of the examples of using with withColumn(), select(), selectExpr() utilizing expr() function.

**CASE** is the start of the expression\
Clause **WHEN** takes a condition, if condition true it returns a value from **THEN**\
If the condition is false it goes to the next condition and so on.\
If none of the condition matches, it returns a value from the ELSE clause.\
**END** is to end the expression.

In [5]:
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]

columns = ["name","gender","salary"]

emp_df = spark.createDataFrame(data, schema = columns)
emp_df.show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|     M| 60000|
|Michael|     M| 70000|
| Robert|  null|400000|
|  Maria|     F|500000|
|    Jen|      |  null|
+-------+------+------+



In [12]:
from pyspark.sql.functions import expr
df1 = emp_df.withColumn("gender", expr("CASE WHEN gender = 'M' THEN 'Male' " +
                                       "WHEN gender = 'F' THEN 'Female' " +
                                       "WHEN gender is Null THEN ' ' "+
                                       "ELSE gender END" ))


df1.show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|  Male| 60000|
|Michael|  Male| 70000|
| Robert|      |400000|
|  Maria|Female|500000|
|    Jen|      |  null|
+-------+------+------+



In [15]:
records = [('Shiva','IT','Bengaluru'),
           ('Rohit','BA','Pune'),
           ('Shubham','IT','Bengaluru'),
           ('Gautam','IT','Chennai')]

Header = ['name','domain','location']

df2 = spark.createDataFrame(records,Header)
df2.show()

+-------+------+---------+
|   name|domain| location|
+-------+------+---------+
|  Shiva|    IT|Bengaluru|
|  Rohit|    BA|     Pune|
|Shubham|    IT|Bengaluru|
| Gautam|    IT|  Chennai|
+-------+------+---------+



In [17]:
df3 = df2.withColumn("role", expr("CASE  WHEN domain = 'IT' THEN 'Software Engineer' " +
                                  "WHEN domain = 'BA' THEN 'Business Analyst' "+
                                  "ELSE domain END"))
df3.show()

+-------+------+---------+-----------------+
|   name|domain| location|             role|
+-------+------+---------+-----------------+
|  Shiva|    IT|Bengaluru|Software Engineer|
|  Rohit|    BA|     Pune| Business Analyst|
|Shubham|    IT|Bengaluru|Software Engineer|
| Gautam|    IT|  Chennai|Software Engineer|
+-------+------+---------+-----------------+



**Multiple Conditions using & and | operator**\
We often need to check with multiple conditions, below is an example of using PySpark When Otherwise with multiple conditions by using and (&) or (|) operators.

**Complete Example – PySpark When Otherwise | SQL Case When**

In [20]:

data = [("James","M",60000), ("Michael","M",70000),
        ("Robert",None,400000), ("Maria","F",500000),
        ("Jen","",None)]

columns = ["name","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

#Using When otherwise
from pyspark.sql.functions import when,col
df2 = df.withColumn("new_gender", when(df.gender == "M","Male")
                                 .when(df.gender == "F","Female")
                                 .when(df.gender.isNull() ,"")
                                 .otherwise(df.gender))
df2.show()

df2=df.select(col("*"),when(df.gender == "M","Male")
                  .when(df.gender == "F","Female")
                  .when(df.gender.isNull() ,"")
                  .otherwise(df.gender).alias("new_gender"))
df2.show()
# Using SQL Case When
from pyspark.sql.functions import expr
df3 = df.withColumn("new_gender", expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
          "ELSE gender END"))
df3.show()

df4 = df.select(col("*"), expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
           "ELSE gender END").alias("new_gender"))

df.createOrReplaceTempView("EMP")
spark.sql("select name, CASE WHEN gender = 'M' THEN 'Male' " +
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
              "ELSE gender END as new_gender from EMP").show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|     M| 60000|
|Michael|     M| 70000|
| Robert|  null|400000|
|  Maria|     F|500000|
|    Jen|      |  null|
+-------+------+------+

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  null|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  null|          |
+-------+------+------+----------+

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|     M| 70000|      Male|
| Robert|  null|400000|          |
|  Maria|     F|500000|    Female|
|    Jen|      |  null|          |
+-------+------+------+----------+

+-------+------+------+----------+
|   name|gender|salary|new_gender|
+-------+------+------+----------+
|  James|     M| 60000|      Male|
|Michael|  